# Advanced RAG Evaluation with Hybrid Search & Reranker

이 노트북은 프로젝트의 **실제 검색 로직(Hybrid Search + Reranker + Dual Query)**을 사용하여 RAG 파이프라인의 성능을 평가합니다.
기존의 단순 Vector Search 평가보다 훨씬 정확한 성능 지표를 제공합니다.

In [8]:
# !pip install ragas langchain langchain-openai qdrant-client rank_bm25 sentence-transformers

In [13]:
import os
import sys
from dotenv import load_dotenv

# 프로젝트 루트 경로 추가 (src 모듈 import 위해)
sys.path.append(os.getcwd())

# 환경 변수 로드 (.env 및 .env.local)
load_dotenv(override=True)
load_dotenv('.env.local', override=True)

from src.utils.config import ConfigDB, ConfigLLM, ConfigAPI
from src.retrievals.search_agent import execute_dual_query_search
from langchain_core.documents import Document
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from ragas import evaluate
from ragas.metrics import (
    context_precision,
    context_recall,
    faithfulness,
    answer_relevancy
)

import pandas as pd

from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from langchain_text_splitters import TokenTextSplitter
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough 
from langchain_core.output_parsers import StrOutputParser
from langchain_core.documents import Document
from ragas import evaluate
from ragas.testset import TestsetGenerator
from ragas.metrics import LLMContextRecall, LLMContextPrecisionWithReference, Faithfulness, AnswerRelevancy
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

from src.utils.config import ConfigDB, ConfigLLM
from src.prompts import ANALYSIS_SYSTEM_PROMPT
from src.retrievals.search_agent import execute_dual_query_search



/var/folders/by/sbz15vxn1xz8499rjvlm_ctm0000gn/T/ipykernel_10052/856362602.py:17: DeprecationWarning: Importing context_precision from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import context_precision
  from ragas.metrics import (
/var/folders/by/sbz15vxn1xz8499rjvlm_ctm0000gn/T/ipykernel_10052/856362602.py:17: DeprecationWarning: Importing context_recall from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import context_recall
  from ragas.metrics import (
/var/folders/by/sbz15vxn1xz8499rjvlm_ctm0000gn/T/ipykernel_10052/856362602.py:17: DeprecationWarning: Importing faithfulness from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import faithfulness
  from ragas.metrics import (
/var/

In [10]:
#################################################################
# 1. Advanced Retriever 정의 (Hybrid + Reranker)
#################################################################

def advanced_retriever(query: str):
    """
    src.retrievals.search_agent의 execute_dual_query_search를 사용하는 Retriever 함수
    - Dual Query (한글/영어)
    - Hybrid Search (Vector + Keyword + BM25)
    - Reranking (Cross-Encoder)
    """
    # 검색 실행
    results, info = execute_dual_query_search(query)
    
    # Dict 결과를 LangChain Document 객체로 변환
    documents = []
    for r in results:
        doc = Document(
            page_content=r['content'],
            metadata=r.get('metadata', {})
        )
        # 점수 정보도 메타데이터에 추가
        doc.metadata['score'] = r.get('score', 0)
        doc.metadata['source'] = r.get('source', '')
        documents.append(doc)
        
    return documents

# 테스트
test_docs = advanced_retriever("파인튜닝이 뭐야?")
print(f"검색된 문서 수: {len(test_docs)}")
for i, doc in enumerate(test_docs):
    print(f"[{i+1}] {doc.page_content[:100]}... (Score: {doc.metadata.get('score'):.4f})")

🔄 Reranking 진행 (후보 3개)...
Compressor 결과:
1. page_content='[강의: 1_LLM_Finetuning개요]

LLM 모델 파인튜닝(Fine tuning)  
파운데이션 모델(Foundation 모델)
**파운데이션 모델**(**foundation model**)은 **대규모 데이터**(**텍스트, 이미지, 오디오, 비디오 등**)로 사전 학습(pre-training)된 **범용 인공지능 모델**로, 다양한 하위 작업(Downstream task)를 위한 **파인튜닝**이나 **프롬프트**(**prompt**) 기반의 응답 모델에 적용할 수 있는 모델을 말한다.  
파운데이션 모델의 특징
- **광범위한 대규모 데이터로 사전 학습**
- 대형 언어 모델(LLM)은 방대한 양의 텍스트 데이터로 사전 학습(pre-training)되며, 그 결과 **다양한 주제와 여러 언어에 대한 일반적인 지식과 언어 패턴등 언어에 대한 범용적 지식**을 포함하고 있다.
- 이미지 기반 파운데이션 모델(image-based foundation model) 또한 대규모 이미지·영상 데이터에서 사물, 장면, 물체 구조의 시각적 특징을 학습하여, **개별 사물의 특성(예: 형태, 질감, 맥락)과 물체들이 공유하는 일반적 물리 특성(예: 경계, 입체감, 재질, 공간 관계)을 표현 공간(representation space)에 반영**한다.
- **대규모 모델**
- 파운데이션 모델은 대부분 **Transformer** 구조를 기반(특히 자연어 모델)으로 하며, **수십억에서 수조개의 파라미터** 규모를 가지고 있다.
- **모델의 크기가 클 수록 성능과 표현력이 향상된다.**
- **범용성**
- **특정 작업에 국한 되지 않고**, 다양한 자연어 처리 작업에 적용될 수 있는 기초 역량을 제공한다.
- 일반적인 NLP task인 문장 생성, 문서요약, 번역, 질의응답에서 좋은 성능을 보인다.
- **프롬프트 기반 실행**
- 추가적인 학습(파인튜닝)

In [46]:
llm_context = """당신은 Python 프로그래밍 교육 과정의 **AI 조교(Teaching Assistant)**이자 데이터셋 생성 전문가입니다.
제공된 [강의 자료(Lecture)]와 [Python 공식 문서(RST)]를 기반으로, 학습자 평가를 위한 고품질의 JSON 포맷 질의응답(QA) 데이터셋을 생성하십시오.

생성 시 다음의 **지침(Instruction)**과 답변 스타일(Tone & Style)을 철저히 준수해야 합니다.

---

"""

llm_context += ANALYSIS_SYSTEM_PROMPT

llm_context += """

---

## QA 생성 제약 (Constraints)
- **언어**: 가능하면 한국어로 질문하고 답변하되, 코드 예제는 영어로 작성하십시오.
- **문법**: 문장의 끝은 마침표(.) 등 **구두점을 반드시 표기**하여 완결된 문장으로 작성하십시오.
- **JSON 포맷 엄수**:
  - 출력은 반드시 파싱 가능한 **JSON 배열** 형태여야 합니다 (`[{"user_input": "...", "reference": "..."}, ...]`).
  - **이스케이프 처리**: 본문 내의 큰따옴표("), 백슬래시(\) 등 특수 문자는 반드시 역슬래시(\")를 사용하여 이스케이프 처리하십시오.
  - 소스 텍스트에 JSON 문법을 해치는 요소가 있더라도, 최종 출력은 유효한 JSON이어야 합니다.

---

## QA 생성 출력 예시
[
  {
    "user_input": "파인튜닝의 장점은 무엇인가?",
    "reference": "파인튜닝은 사전 학습된 모델(Foundation 모델)을 특정 태스크나 도메인 데이터로 추가 학습 최적화 하는 과정으로, 특정 작업/도메인에 최적화, 사용자 맞춤형 톤앤매너 적용, 안전성과 윤리 강화를 위해 필요하다."
  }
]

위 가이드를 바탕으로 주어진 텍스트에서 학습 가치가 높은 QA 세트를 생성하십시오.
"""

<>:21: SyntaxWarning: invalid escape sequence '\)'
<>:21: SyntaxWarning: invalid escape sequence '\)'
/var/folders/by/sbz15vxn1xz8499rjvlm_ctm0000gn/T/ipykernel_10052/3359774379.py:21: SyntaxWarning: invalid escape sequence '\)'
  - **이스케이프 처리**: 본문 내의 큰따옴표("), 백슬래시(\) 등 특수 문자는 반드시 역슬래시(\")를 사용하여 이스케이프 처리하십시오.


In [47]:
client = QdrantClient(host='localhost', port=ConfigDB.PORT)
info = client.get_collection(ConfigDB.COLLECTION_NAME)

results, next_id = client.scroll(
    collection_name=ConfigDB.COLLECTION_NAME,
    limit=info.points_count,
)

# sampling
import random
sample_dataset = random.sample(results, 50) # 리스트에서 랜덤하게 K개를 추출


# 문서 내용만 추출
docs = [point.payload['page_content'] for point in sample_dataset if point.payload.get('metadata', {}).get('source', '') == 'lecture']
docs

['[강의: 09_CNN_개요]\n\n1D Convolution 연산',
 '[강의: 11_RAG_evaluation]\n\nRAG 평가 개요\n- RAG 평가란 RAG 시스템이 주어진 입력에 대해 얼마나 효과적으로 관련 정보를 검색하고, 이를 기반으로 정확하고 유의미한 응답을 생성하는지를 측정하는 과정이다.\n- **평가 요소**\n- **검색 단계 평가**\n- 입력 질문에 대해 검색된 문서나 정보의 관련성과 정확성을 평가.\n- **생성 단계 평가**\n- 검색된 정보를 기반으로 생성된 응답의 품질, 정확성등을 평가.\n- **평가 방법**\n- 온/오프라인 평가\n1. **오프라인 평가**\n- 미리 준비된 데이터셋을 활용하여 RAG 시스템의 성능을 측정한다.\n2. **온라인 평가**\n- 실제 사용자 트래픽과 피드백을 기반으로 시스템의 실시간 성능을 평가한다.\n- 정량적/정성적 평가\n1. 정량적 평가\n- 자동화된 지표를 사용하여 생성된 텍스트의 품질을 평가한다.\n2. 정성적 평가\n- 전문가나 일반 사용자가 직접 생성된 응답의 품질을 평가하여 주관적인 지표를 평가한다.',
 '[강의: 02_첫번째 머신러닝 분석 - Iris_분석]\n\n그런데 이 결과가 맞을까?',
 '[강의: 14 군집_Clustering]\n\nKMeans\n- sklearn.cluster.KMeans\n- 하이퍼파라미터\n- n_clusters: 몇개의 category로 분류할 지 지정.\n- 속성\n- labels_ : 데이터포인트별 label',
 '[강의: 09_CNN_개요]\n\nStrides\n- Filter(Kernel)가 한번 Convolution 연산을 수행한 후 옆 혹은 아래로 얼마나 이동할 것인가를 설정.\n- 값으로는 다음 두가지 타입을 지정할 수 있다.\n- tuple: (height방향 이동크기, width 방향 이동크기)\n- int: height, width 방향 이동크기가 같은 경우 정수값 하나로 설정한다.\n- 예) stride=2: 한 번

In [48]:
#################################################################
# 2. Testset 준비
# (기존에 생성해둔 평가 데이터셋을 사용하거나 새로 생성)
#################################################################
import pandas as pd
from datasets import Dataset

# 예시 데이터셋 (실제로는 TestsetGenerator로 생성한 데이터를 사용하세요)
# 여기서는 사용자님이 이전에 생성했던 데이터프레임 구조를 가정합니다.
# 만약 저장된 데이터가 있다면 pd.read_csv() 등을 사용하세요.

generator_llm = LangchainLLMWrapper(ChatOpenAI(model=ConfigLLM.OPENAI_MODEL))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model=ConfigDB.EMBEDDING_MODEL))

generator = TestsetGenerator(
    llm=generator_llm,
    embedding_model=generator_embeddings,
    llm_context=llm_context 
)

# testset
testset = generator.generate_with_chunks(
    docs, testset_size=20  # context 내용, 테스트데이터셋 몇개를 만들지.
)
testset


/var/folders/by/sbz15vxn1xz8499rjvlm_ctm0000gn/T/ipykernel_10052/3796546008.py:12: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  generator_llm = LangchainLLMWrapper(ChatOpenAI(model=ConfigLLM.OPENAI_MODEL))
/var/folders/by/sbz15vxn1xz8499rjvlm_ctm0000gn/T/ipykernel_10052/3796546008.py:13: DeprecationWarning: LangchainEmbeddingsWrapper is deprecated and will be removed in a future version. Use the modern embedding providers instead: embedding_factory('openai', model='text-embedding-3-small', client=openai_client) or from ragas.embeddings import OpenAIEmbeddings, GoogleEmbeddings, HuggingFaceEmbeddings
  generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model=ConfigDB.EMBEDDING_MODEL))
Applying CustomNodeFilter:   0%|          | 0/9 [00:00<?, ?it/s]Node efdc7d68-c8ca-4ee4-

Testset(samples=[TestsetSample(eval_sample=SingleTurnSample(user_input='정성적 평가란 무엇인가요?', retrieved_contexts=None, reference_contexts=['[강의: 11_RAG_evaluation]\n\nRAG 평가 개요\n- RAG 평가란 RAG 시스템이 주어진 입력에 대해 얼마나 효과적으로 관련 정보를 검색하고, 이를 기반으로 정확하고 유의미한 응답을 생성하는지를 측정하는 과정이다.\n- **평가 요소**\n- **검색 단계 평가**\n- 입력 질문에 대해 검색된 문서나 정보의 관련성과 정확성을 평가.\n- **생성 단계 평가**\n- 검색된 정보를 기반으로 생성된 응답의 품질, 정확성등을 평가.\n- **평가 방법**\n- 온/오프라인 평가\n1. **오프라인 평가**\n- 미리 준비된 데이터셋을 활용하여 RAG 시스템의 성능을 측정한다.\n2. **온라인 평가**\n- 실제 사용자 트래픽과 피드백을 기반으로 시스템의 실시간 성능을 평가한다.\n- 정량적/정성적 평가\n1. 정량적 평가\n- 자동화된 지표를 사용하여 생성된 텍스트의 품질을 평가한다.\n2. 정성적 평가\n- 전문가나 일반 사용자가 직접 생성된 응답의 품질을 평가하여 주관적인 지표를 평가한다.'], retrieved_context_ids=None, reference_context_ids=None, response=None, multi_responses=None, reference='정성적 평가는 전문가나 일반 사용자가 직접 생성된 응답의 품질을 평가하여 주관적인 지표를 평가하는 과정입니다.', rubrics=None, persona_name='Data Science Educator', query_style='WEB_SEARCH_LIKE', query_length='SHORT'), synthesizer_name='single_hop_specific_query_synthesizer'), TestsetSampl

In [49]:
eval_df = testset.to_pandas()

# Reference 데이터 재생성 (ANALYSIS_SYSTEM_PROMPT 적용)
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from src.prompts import ANALYSIS_SYSTEM_PROMPT

# LLM 설정
gt_llm = ChatOpenAI(model=ConfigLLM.OPENAI_MODEL, temperature=0)
gt_prompt = ChatPromptTemplate.from_template(ANALYSIS_SYSTEM_PROMPT)
gt_chain = gt_prompt | gt_llm | StrOutputParser()

print("Reference 데이터 재생성 중...")
new_references = []
for idx, row in eval_df.iterrows():
    if isinstance(row['reference_contexts'], list):
        context_str = "\n\n".join(row['reference_contexts'])
    else:
        context_str = str(row['reference_contexts'])
    
    query = row['user_input']
    
    # Generate
    try:
        new_ref = gt_chain.invoke({"context": context_str, "query": query})
    except Exception as e:
        print(f"Error generating reference for query {query}: {e}")
        new_ref = row.get('reference', '') # Fallback to original
        
    new_references.append(new_ref)

eval_df['reference'] = new_references

# 다음 셀 호환성을 위한 데이터 준비
eval_data = eval_df.rename(columns={'user_input': 'question'}).to_dict('list')

eval_df.head()

Reference 데이터 재생성 중...


,user_input,reference_contexts,reference,persona_name,query_style,query_length,synthesizer_name
0,정성적 평가란 무엇인가요?,[[강의: 11_RAG_evaluation]\n\nRAG 평가 개요\n- RAG 평...,**핵심 답변**: 정성적 평가는 전문가나 일반 사용자가 직접 생성된 응답의 품질을...,Data Science Educator,WEB_SEARCH_LIKE,SHORT,single_hop_specific_query_synthesizer
1,RAG 시스템의 평가 방법은 어떤 것들이 있나요?,[[강의: 11_RAG_evaluation]\n\nRAG 평가 개요\n- RAG 평...,**핵심 답변**: RAG 시스템의 평가 방법은 오프라인 평가와 온라인 평가로 나뉘...,Data Science Student,POOR_GRAMMAR,LONG,single_hop_specific_query_synthesizer
2,오프라인 평가가 뭐에요?,[[강의: 11_RAG_evaluation]\n\nRAG 평가 개요\n- RAG 평...,**핵심 답변**: 오프라인 평가는 미리 준비된 데이터셋을 활용하여 RAG 시스템의...,Data Science Student,POOR_GRAMMAR,MEDIUM,single_hop_specific_query_synthesizer
3,What are the key attributes of sklearn.cluster...,[[강의: 14 군집_Clustering]\n\nKMeans\n- sklearn.c...,**핵심 답변**: `sklearn.cluster.KMeans`의 주요 속성은 `l...,Data Science Student,PERFECT_GRAMMAR,MEDIUM,single_hop_specific_query_synthesizer
4,sklearn.cluster.KMeans의 하이퍼파라미터 n_clusters는 무엇...,[[강의: 14 군집_Clustering]\n\nKMeans\n- sklearn.c...,**핵심 답변**: `sklearn.cluster.KMeans`의 하이퍼파라미터 `...,Data Evaluation Analyst,MISSPELLED,LONG,single_hop_specific_query_synthesizer


In [50]:
#################################################################
# 3. RAG 파이프라인 구성 및 실행
#################################################################
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from src.prompts import ANALYSIS_SYSTEM_PROMPT
from langchain_core.runnables import RunnableLambda

# LLM 설정
llm = ChatOpenAI(model=ConfigLLM.OPENAI_MODEL, temperature=0)

# 프롬프트
prompt = ChatPromptTemplate.from_template(ANALYSIS_SYSTEM_PROMPT)

# 문서 포맷팅
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain 구성
rag_chain = (
    {
        "context": RunnableLambda(advanced_retriever) | RunnableLambda(format_docs), 
        "query": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)

# 답변 생성 및 Context 수집
answers = []
contexts = []

print("평가 데이터에 대한 응답 생성 중...")
for q in eval_data['question']:
    # 1. 문서 검색
    retrieved_docs = advanced_retriever(q)
    context_text = [doc.page_content for doc in retrieved_docs]
    contexts.append(context_text)
    
    # 2. 답변 생성
    # Chain을 직접 호출하면 context를 재검색하므로(retriever가 chain 안에 있음),
    # 효율성을 위해 미리 검색한 docs를 주입하거나, 그냥 chain을 invoke합니다.
    # 여기서는 정확한 평가를 위해 chain invoke 결과를 사용합니다.
    response = rag_chain.invoke(q)
    answers.append(response)
    # print(f"Q: {q}\nA: {response[:50]}...\n")

# 데이터셋에 결과 추가
eval_data['answer'] = answers
eval_data['contexts'] = contexts

final_dataset = Dataset.from_dict(eval_data)

평가 데이터에 대한 응답 생성 중...
🔄 Reranking 진행 (후보 3개)...
Compressor 결과:
1. page_content='[강의: 11_RAG_evaluation]

RAG 평가 개요
- RAG 평가란 RAG 시스템이 주어진 입력에 대해 얼마나 효과적으로 관련 정보를 검색하고, 이를 기반으로 정확하고 유의미한 응답을 생성하는지를 측정하는 과정이다.
- **평가 요소**
- **검색 단계 평가**
- 입력 질문에 대해 검색된 문서나 정보의 관련성과 정확성을 평가.
- **생성 단계 평가**
- 검색된 정보를 기반으로 생성된 응답의 품질, 정확성등을 평가.
- **평가 방법**
- 온/오프라인 평가
1. **오프라인 평가**
- 미리 준비된 데이터셋을 활용하여 RAG 시스템의 성능을 측정한다.
2. **온라인 평가**
- 실제 사용자 트래픽과 피드백을 기반으로 시스템의 실시간 성능을 평가한다.
- 정량적/정성적 평가
1. 정량적 평가
- 자동화된 지표를 사용하여 생성된 텍스트의 품질을 평가한다.
2. 정성적 평가
- 전문가나 일반 사용자가 직접 생성된 응답의 품질을 평가하여 주관적인 지표를 평가한다.' metadata={'source': '', 'source_file': '11_RAG_evaluation.ipynb', 'lecture_title': '11_RAG_evaluation', 'cell_type': 'markdown', 'cell_index': 0, 'code_snippet': '```python\n!uv pip install ragas rapidfuzz\n```', 'chunk_index': 0, 'original_score': 0.6524498320000001}
   점수: 0.0
   원본 점수: 0.6524498320000001
--------------------------------------------------
2. page_content='[강의: 11_RAG_evaluation]

RAGAS 평가 실습' metada

In [51]:
#################################################################
# 4. Ragas 평가 실행
#################################################################

result = evaluate(
    final_dataset,
    metrics=[
        context_precision,
        context_recall,
        faithfulness,
        answer_relevancy,
    ],
    llm=llm,
    embeddings=OpenAIEmbeddings(model=ConfigDB.EMBEDDING_MODEL)
)

print("\n========== 평가 결과 ==========")
print(result)
df_result = result.to_pandas()
df_result

Evaluating:  52%|█████▏    | 31/60 [00:57<00:42,  1.47s/it]LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
Evaluating: 100%|██████████| 60/60 [01:38<00:00,  1.64s/it]



========== 평가 결과 ==========
{'context_precision': 0.8333, 'context_recall': 0.7044, 'faithfulness': 0.8240, 'answer_relevancy': 0.5833}


,user_input,retrieved_contexts,reference_contexts,response,reference,persona_name,query_style,query_length,context_precision,context_recall,faithfulness,answer_relevancy
0,정성적 평가란 무엇인가요?,[[강의: 11_RAG_evaluation]\n\nRAG 평가 개요\n- RAG 평...,[[강의: 11_RAG_evaluation]\n\nRAG 평가 개요\n- RAG 평...,**핵심 답변**: 정성적 평가는 전문가나 일반 사용자가 직접 생성된 응답의 품질을...,**핵심 답변**: 정성적 평가는 전문가나 일반 사용자가 직접 생성된 응답의 품질을...,Data Science Educator,WEB_SEARCH_LIKE,SHORT,0.833333,1.000000,1.000000,0.344786
1,RAG 시스템의 평가 방법은 어떤 것들이 있나요?,[[강의: 11_RAG_evaluation]\n\nRAG 평가 개요\n- RAG 평...,[[강의: 11_RAG_evaluation]\n\nRAG 평가 개요\n- RAG 평...,RAG 시스템의 평가 방법은 크게 두 가지로 나눌 수 있습니다: **오프라인 평가*...,**핵심 답변**: RAG 시스템의 평가 방법은 오프라인 평가와 온라인 평가로 나뉘...,Data Science Student,POOR_GRAMMAR,LONG,1.000000,1.000000,1.000000,0.796023
2,오프라인 평가가 뭐에요?,[[강의: 11_RAG_evaluation]\n\nRAG 평가 개요\n- RAG 평...,[[강의: 11_RAG_evaluation]\n\nRAG 평가 개요\n- RAG 평...,**핵심 답변**: 오프라인 평가는 실제 환경에서 학생의 학습 성과를 평가하기 위해...,**핵심 답변**: 오프라인 평가는 미리 준비된 데이터셋을 활용하여 RAG 시스템의...,Data Science Student,POOR_GRAMMAR,MEDIUM,1.000000,1.000000,0.000000,0.624273
3,What are the key attributes of sklearn.cluster...,[[강의: 14 군집_Clustering]\n\nKMeans\n- sklearn.c...,[[강의: 14 군집_Clustering]\n\nKMeans\n- sklearn.c...,**핵심 답변**: `sklearn.cluster.KMeans`의 주요 속성은 `n...,**핵심 답변**: `sklearn.cluster.KMeans`의 주요 속성은 `l...,Data Science Student,PERFECT_GRAMMAR,MEDIUM,0.833333,1.000000,1.000000,0.849916
4,sklearn.cluster.KMeans의 하이퍼파라미터 n_clusters는 무엇...,[[강의: 14 군집_Clustering]\n\nKMeans\n- sklearn.c...,[[강의: 14 군집_Clustering]\n\nKMeans\n- sklearn.c...,**핵심 답변**: `sklearn.cluster.KMeans`의 하이퍼파라미터 `...,**핵심 답변**: `sklearn.cluster.KMeans`의 하이퍼파라미터 `...,Data Evaluation Analyst,MISSPELLED,LONG,0.833333,1.000000,1.000000,0.770000
5,sklearn.cluster.KMeans는 무엇인가요?,[[강의: 14 군집_Clustering]\n\nKMeans\n- sklearn.c...,[[강의: 14 군집_Clustering]\n\nKMeans\n- sklearn.c...,**핵심 답변**: `sklearn.cluster.KMeans`는 K-평균 군집화 ...,**핵심 답변**: `sklearn.cluster.KMeans`는 K-평균 군집화 ...,Data Science Educator,MISSPELLED,SHORT,1.000000,0.800000,1.000000,0.706331
6,tensor 어떻게 생성하나요?,[[KEYWORDS] array floor division\n[TITLE] arra...,[[강의: 02. tensor 다루기]\n\n빈 tensor 생성\n- **torc...,핵심 답변: 강의 자료에서는 텐서 생성에 대한 내용이 직접적으로 다뤄지지 않습니다....,**핵심 답변**: tensor는 `torch.empty(*size)` 함수를 사용...,Data Science Educator,MISSPELLED,SHORT,0.000000,0.000000,0.428571,0.000000
7,How can I create an empty tensor using torch?,[[TITLE] typing\n[H1] Annotating tuples\n\n# E...,[[강의: 02. tensor 다루기]\n\n빈 tensor 생성\n- **torc...,강의 자료에서는 빈 텐서를 생성하는 방법에 대한 내용이 직접적으로 다뤄지지 않습니다...,**핵심 답변**: `torch.empty(*size)` 함수를 사용하여 빈 ten...,Data Science Educator,PERFECT_GRAMMAR,SHORT,0.000000,0.000000,0.400000,0.000000
8,tensor 어떻게 만들어요?,[[강의: 02. tensor 다루기]\n\n원하는 형태(shape) 텐서 생성\n...,[[강의: 02. tensor 다루기]\n\n빈 tensor 생성\n- **torc...,"텐서를 만들기 위해서는 `torch.tensor(자료구조 [, dtype])` 함수...",**핵심 답변**: tensor는 `torch.empty(*size)` 함수를 사용...,Data Evaluation Analyst,POOR_GRAMMAR,SHORT,1.000000,0.000000,1.000000,0.529952
9,회귀는 지도학습에서 어떤 역할을 하나요?,[[강의: 01_머신러닝개요]\n\n- ### 분류(Classification):\...,[[강의: 01_머신러닝개요]\n\n머신러닝 알고리즘 분류 \n지도학습(Super...,"회귀는 지도학습에서 **숫자(연속된 값)를 예측하는 역할**을 합니다. 즉, 주어진...","회귀는 지도학습에서 연속적인 값을 예측하는 역할을 합니다. 즉, 주어진 데이터의 특...",Data Science Student,WEB_SEARCH_LIKE,SHORT,1.000000,0.750000,0.857143,0.999998
